<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Differential_Privacy_for_Secure_Data_Sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opacus

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from opacus import PrivacyEngine
from torch.utils.data import DataLoader, TensorDataset

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Create a dataset and dataloader for demonstration purposes
inputs = torch.randn(1024, 10)
labels = torch.randint(0, 2, (1024,))
dataset = TensorDataset(inputs, labels)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training with differential privacy
model = SimpleNet(input_dim=10, output_dim=2)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Initialize Privacy Engine without parameters
privacy_engine = PrivacyEngine()

# Attach Privacy Engine to the optimizer, model, and dataloader
model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=1.1,
    max_grad_norm=1.0,
)

# Dummy training loop with privacy guarantees
for epoch in range(5):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Calculate privacy spent using the accountant
epsilon = privacy_engine.accountant.get_epsilon(delta=1e-5)
print(f"Model trained with differential privacy, ε: {epsilon:.2f}")